<a href="https://colab.research.google.com/github/mhmdghazal/Install-Caffe-using-Colab-/blob/master/Caffe_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build CAFFE CPU-ONLY IN COLAB

(In case anything goes wrong)

In [0]:
#!kill -9 -1

In [2]:
!ls

sample_data


main commands needed before install caffe, it will take some time

In [3]:
print "Install deps and gcc"

with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))
  
t=!apt update
t=!apt upgrade
t=!sudo apt-get install -y build-essential cmake git pkg-config libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libatlas-base-dev libgflags-dev libgoogle-glog-dev liblmdb-dev  python3-dev python-numpy python-scipy libopencv-dev liblapack-dev liblapack3 libopenblas-base libopenblas-dev 
t=!sudo apt-get install -y --no-install-recommends libboost-all-dev

print "Get boost"

t=!wget https://dl.bintray.com/boostorg/release/1.67.0/source/boost_1_67_0.tar.bz2
t=!tar --bzip2 -xf boost_1_67_0.tar.bz2

print "Compile boost"

t=!cd boost_1_67_0 && ./bootstrap.sh --exec-prefix=/usr/local --with-libraries=system,filesystem,regex,thread,python \
--with-python-version=2.7 --with-python-root=/usr
t=!cd boost_1_67_0 && ./b2 install
t=!sudo apt-get install libgflags-dev
t=!sudo apt-get install protobuf





Install deps and gcc
Get boost
Compile boost


Install Caffe for CPU-ONLY 

In [4]:
print "Clone Caffe and configure it"

t=!git clone https://github.com/weiliu89/caffe.git && cd caffe && git checkout ssd
  
with open('caffe/Makefile.config.example') as f:
  config = f.read()
comment = ['BLAS := open']
uncomment = ['# CPU_ONLY := 1','# BLAS := atlas', 
             '# OPENCV_VERSION := 3', '# WITH_PYTHON_LAYER := 1'] #
replace = [('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include',
            'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial /usr/local/lib/python2.7/dist-packages/numpy/core/include/'), 
           ('LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib',
            'LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial')]
for c in uncomment:
  config = config.replace(c, c[2:])
for c in comment:
  config = config.replace(c, '# '+c)
for c1,c2 in replace:
  config = config.replace(c1, c2)
with open('caffe/Makefile.config', 'w') as f:
  f.write(config)
  
with open('caffe/Makefile') as f:
  mfile = f.read()
with open('caffe/Makefile', 'w') as f:
  f.write(mfile.replace('-isystem','-I'))
  
print "Make Caffe"

#t=!cd caffe && make -j8 && make pycaffe && make test -j8 && make distribute

t=!cd caffe && make clean && make -j8 && make pycaffe && make distribute
print '\n'.join(t[-10:])

t=!echo /usr/local/lib >> /etc/ld.so.conf && ldconfig
t=!echo /content/caffe/distribute/lib >> /etc/ld.so.conf && ldconfig

print "DONE"

Clone Caffe and configure it
Make Caffe
cp .build_release/src/caffe/proto/caffe.pb.h distribute/include/caffe/proto
# add tool and example binaries
cp .build_release/tools/device_query.bin .build_release/tools/caffe.bin .build_release/tools/train_net.bin .build_release/tools/upgrade_net_proto_text.bin .build_release/tools/upgrade_solver_proto_text.bin .build_release/tools/test_net.bin .build_release/tools/upgrade_net_proto_binary.bin .build_release/tools/get_image_size.bin .build_release/tools/net_speed_benchmark.bin .build_release/tools/convert_imageset.bin .build_release/tools/extract_features.bin .build_release/tools/finetune_net.bin .build_release/tools/create_label_map.bin .build_release/tools/convert_annoset.bin .build_release/tools/compute_image_mean.bin distribute/bin
cp .build_release/examples/cifar10/convert_cifar_data.bin .build_release/examples/ssd/ssd_detect.bin .build_release/examples/siamese/convert_mnist_siamese_data.bin .build_release/examples/mnist/convert_mnist_data.

Import Caffe for test

In [0]:
#test caffe
import sys
import os
caffe_path = !cd /content/caffe/python
sys.path.insert(0, '/content/caffe/python')  
import caffe

import cv2
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

from caffe.proto.caffe_pb2 import SolverParameter
import google.protobuf.text_format as txtf